# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [1]:
# Подключим сразу все, что нам пригодится
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# считаем файл
df = pd.read_csv('/datasets/users_behavior.csv')

# посмотрим описание данных
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [2]:
# Посмотрим info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
# Посмотрим
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
# Хоть в рамках задания обозначено, что предобработку мы уже выполнили, типы messages и calls
# явно не соотетствуют действительности, приведем их к int

df['calls'] = df['calls'].astype('int')
df['messages'] = df['messages'].astype('int')

#### Промежуточный вывод

С данными никаких преобразований не понадобилось. Согласно условию, они изначально валидны, пропусков нет.
В нашем случае достаточно было изменить тип соответствующих столбцов

## 2. Разбейте данные на выборки

In [5]:
# для начала определим наш random_state, который будем использовать для всех методов
RANDOM_STATE = 125

# т.к. нам нужны три выборки, будем разбивать на три группы в 60-20-20
# сначала отделим набор на котором будем тренировать
df_train, df_valid_ant_test = train_test_split(df, test_size=0.6, random_state=RANDOM_STATE)
# теперь разделим тестовую выборку от валидационной
df_valid, df_test = train_test_split(df_valid_ant_test, test_size=0.5, random_state=RANDOM_STATE)
del df_valid_ant_test

## 3. Исследуйте модели

#### Ремарка

Чтобы не искать глазами какая же модель и при каких параметрах дала лучший результат, определим для этого соответствующую переменную и если какая-то модель даст лучшую точность - будем ее перезаписывать в реальном времени.

In [6]:
# Определим переменную для сохранения лучших параметров модели
best_accuracy_dict = dict()

# функция, чтобы меньше дублировать код в циклах
def fill_best_accuracy_dict(model, modelName, accuracy):
    best_accuracy_dict['model'] = model
    best_accuracy_dict['modelName'] = modelName
    best_accuracy_dict['accuracy'] = accuracy
    return best_accuracy_dict

# значения по умолчанию
best_accuracy_dict = fill_best_accuracy_dict('Undefined', 'Undefined', 0)

In [7]:
# Для начала определим наши признаки

# тренировочные
train_features = df_train.drop(['is_ultra'], axis=1)
train_target = df_train['is_ultra']
# валидационные
valid_features = df_valid.drop(['is_ultra'], axis=1)
valid_target = df_valid['is_ultra']
# тестовые
test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']

In [8]:
# исследуем дерево решений
for max_depth in range (2, 11, 1):
    modelDecisionTree = DecisionTreeClassifier(max_depth=max_depth, random_state=RANDOM_STATE)
    modelDecisionTree.fit(train_features, train_target)
    valid_predicted = modelDecisionTree.predict(valid_features)
    
    accuracy = accuracy_score(valid_target, valid_predicted)
    
    modelName = 'Descision Tree; max_depth= ' + str(max_depth)
    if (accuracy > best_accuracy_dict['accuracy']):
        best_accuracy_dict = fill_best_accuracy_dict(modelDecisionTree, modelName, accuracy)
    print(modelName,'; accuracy =', accuracy)

Descision Tree; max_depth= 2 ; accuracy = 0.7780082987551867
Descision Tree; max_depth= 3 ; accuracy = 0.7894190871369294
Descision Tree; max_depth= 4 ; accuracy = 0.7811203319502075
Descision Tree; max_depth= 5 ; accuracy = 0.7842323651452282
Descision Tree; max_depth= 6 ; accuracy = 0.7842323651452282
Descision Tree; max_depth= 7 ; accuracy = 0.7759336099585062
Descision Tree; max_depth= 8 ; accuracy = 0.7759336099585062
Descision Tree; max_depth= 9 ; accuracy = 0.7572614107883817
Descision Tree; max_depth= 10 ; accuracy = 0.7769709543568465


In [9]:
# исследуем случайный лес
for estim in range(5, 51, 5):
    for max_depth in range(5, 31, 5):
        modelRandomForest = RandomForestClassifier(n_estimators=estim, max_depth=max_depth, random_state=RANDOM_STATE)
        modelRandomForest.fit(train_features, train_target)
        valid_predicted = modelRandomForest.predict(valid_features)
        
        accuracy = accuracy_score(valid_target, valid_predicted)
        
        modelName = "Random Forest; n_estimators = " + str(estim) + " ; max_depth= " + str(max_depth)
        if (accuracy > best_accuracy_dict['accuracy']):
            best_accuracy_dict = fill_best_accuracy_dict(modelRandomForest, modelName, accuracy)
        print(modelName,'; accuracy =', accuracy)
    print() # для разделения вывода

Random Forest; n_estimators = 5 ; max_depth= 5 ; accuracy = 0.7914937759336099
Random Forest; n_estimators = 5 ; max_depth= 10 ; accuracy = 0.779045643153527
Random Forest; n_estimators = 5 ; max_depth= 15 ; accuracy = 0.774896265560166
Random Forest; n_estimators = 5 ; max_depth= 20 ; accuracy = 0.7354771784232366
Random Forest; n_estimators = 5 ; max_depth= 25 ; accuracy = 0.7531120331950207
Random Forest; n_estimators = 5 ; max_depth= 30 ; accuracy = 0.7531120331950207

Random Forest; n_estimators = 10 ; max_depth= 5 ; accuracy = 0.7956431535269709
Random Forest; n_estimators = 10 ; max_depth= 10 ; accuracy = 0.7873443983402489
Random Forest; n_estimators = 10 ; max_depth= 15 ; accuracy = 0.7780082987551867
Random Forest; n_estimators = 10 ; max_depth= 20 ; accuracy = 0.770746887966805
Random Forest; n_estimators = 10 ; max_depth= 25 ; accuracy = 0.779045643153527
Random Forest; n_estimators = 10 ; max_depth= 30 ; accuracy = 0.779045643153527

Random Forest; n_estimators = 15 ; max_

In [10]:
# уберем будущий ворнинг
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Ислледуем логистическую регрессию
modelLogisticRegression = LogisticRegression(random_state=RANDOM_STATE)
modelLogisticRegression.fit(train_features, train_target)
valid_predicted = modelLogisticRegression.predict(valid_features)

accuracy = accuracy_score(valid_target, valid_predicted)
        
modelName = 'Logistic Regression'
if (accuracy > best_accuracy_dict['accuracy']):
    best_accuracy_dict = fill_best_accuracy_dict(modelLogisticRegression, modelName, accuracy)
print(modelName,'; accuracy =', accuracy)

Logistic Regression ; accuracy = 0.7085062240663901


In [11]:
# Посмотрим, какая модель оказалась лучше всего
best_accuracy_dict

{'model': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=10, max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=25,
                        n_jobs=None, oob_score=False, random_state=125,
                        verbose=0, warm_start=False),
 'modelName': 'Random Forest; n_estimators = 25 ; max_depth= 10',
 'accuracy': 0.8060165975103735}

#### Промежуточный вывод

- Лучший результат в точности показал случайный лес с параметрами n_estimators = 25 ; max_depth= 10
- Худший результат в точночти показала логистическая регрессия
- Довольно неплохо показало себя решающее дерево, при чем глубины 3 было достаточно
- Из любопытного - независимо от числа деревьев, лучшие результаты показывал лес с глубиной 10

## 4. Проверьте модель на тестовой выборке

In [12]:
# проверим модель на тестовых данных
model = best_accuracy_dict['model']
test_predicted = model.predict(test_features)
print('Test accuracy =', accuracy_score(test_target, test_predicted))

Test accuracy = 0.8031088082901554


In [13]:
# из интереса проверим точность модели на исходных данных
train_predicted = model.predict(train_features)
print('Train accuracy =', accuracy_score(train_target, train_predicted))

Train accuracy = 0.8894941634241245


#### Предварительный вывод

Мы получили хорошую точность модели. При этом, учитывая, что на исходных данных не ушли за 90%, можно оценить фактор переобучения как не сильно критичный для нашей модели.

## 5. (бонус) Проверьте модели на адекватность

In [14]:
# Проверим модель на адекватность
# В задаче классификации для этих целей необходимо создать случайный набор предсказаний

import random
random_predictions = pd.Series(test_target.apply(lambda x: round(random.random())), index=test_target.index)

# оценим точность на этих данных
print('Random predictions accuracy = ', accuracy_score(test_target, random_predictions))

Random predictions accuracy =  0.48704663212435234


## Итоговый вывод

Мы провели хоть и небольшую, но отличную работу.
- Мы изучили наши данные и привели типы к нужным значениям
- Создали три выборки по которым обучили и проверили нашу модель в пропорции 60% - 20% - 20%
- Мы исследовали три различные модели с разными значениями гиперпараметров (Дерево решений, Случайный лес, Логистическую регрессию)
- Мы определили лучшую модель и параметры для нее, измерив и сравнив точность наших предсказаний на валидационной выборке (в нашем случае лучший результат показал случайный лес с 25 деревьями и глубиной 10)
- Мы проверили нашу модель на тестовых данных и они оказались очень близко к данным на валидной выборке. При этом переобучение для нашей модели присутствует, но не критично.
- Дополнительно мы проверили модель на адекватность и наша модель показала хорошие результаты в сравнении со случайными предсказаниями.

Теперь "Мегалайн" может спать спокойно, т.к. у него есть модель, которая позволит предлагать изменение тарифа заинтересованным в этом клиентам (по крайней мере с вероятностью 80%+)